## fasttect word embedding facebook library

### genism의 fasttext와 다른점.
* facebook fasttext library는 supervised classifier도 제공한다.

### Reference
* lovit님의 fasttext tutorial
* https://fasttext.cc/blog/2019/06/25/blog-post.html#2-you-were-using-the-unofficial-fasttext-module

### dataset
* 네이버 영화 댓글

In [9]:
import soynlp
import fasttext
import pickle

### import data

In [51]:
path = 'ratings_train.txt'
with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
        
naver_comments = [i.split('\t')[1] for i in lines][1:]
fasttext_corpus = [ encode(i).split(' ') for i in naver_comments]
labels = [i.split('\t')[2].split('\n')[0] for i in lines][1:]
labels = [int(i) for i in labels]

### to make data format corresponding to fasttext

In [58]:
num2label = {0:'neg', 1:'pos'}
train = ['__label__' + num2label[labels[i]] + ' ' + ' '.join(fasttext_corpus[i]) for i in range(len(labels)) ]
train[:5]

['__label__neg ㅇㅏ- ㄷㅓ-ㅂㅣㅇ ㅈㅣㄴㅉㅏ- ㅉㅏ-ㅈㅡㅇㄴㅏ-ㄴㅔ-ㅇㅛ- ㅁㅗㄱㅅㅗ-ㄹㅣ-',
 '__label__pos ㅎㅡㅁ ㅍㅗ-ㅅㅡ-ㅌㅓ-ㅂㅗ-ㄱㅗ- ㅊㅗ-ㄷㅣㅇㅇㅕㅇㅎㅘ-ㅈㅜㄹ ㅇㅗ-ㅂㅓ-ㅇㅕㄴㄱㅣ-ㅈㅗ-ㅊㅏ- ㄱㅏ-ㅂㅕㅂㅈㅣ- ㅇㅏㄶㄱㅜ-ㄴㅏ-',
 '__label__neg ㄴㅓ-ㅁㅜ-ㅈㅐ-ㅁㅣㅎㅇㅓㅆㄷㅏ-ㄱㅡ-ㄹㅐ-ㅅㅓ-ㅂㅗ-ㄴㅡㄴㄱㅓㅅㅇㅡㄹㅊㅜ-ㅊㅓㄴㅎㅏㄴㄷㅏ-',
 '__label__neg ㄱㅛ-ㄷㅗ-ㅅㅗ- ㅇㅣ-ㅇㅑ-ㄱㅣ-ㄱㅜ-ㅁㅓㄴ ㅅㅗㄹㅈㅣㄱㅎㅣ- ㅈㅐ-ㅁㅣ-ㄴㅡㄴ ㅇㅓㅄㄷㅏ- ㅍㅕㅇㅈㅓㅁ ㅈㅗ-ㅈㅓㅇ',
 '__label__pos ㅅㅏ-ㅇㅣ-ㅁㅗㄴㅍㅔ-ㄱㅡ-ㅇㅢ- ㅇㅣㄱㅅㅏㄹㅅㅡ-ㄹㅓㄴ ㅇㅕㄴㄱㅣ-ㄱㅏ- ㄷㅗㄷㅂㅗ-ㅇㅕㅆㄷㅓㄴ ㅇㅕㅇㅎㅘ- ㅅㅡ-ㅍㅏ-ㅇㅣ-ㄷㅓ-ㅁㅐㄴㅇㅔ-ㅅㅓ- ㄴㅡㄺㅇㅓ-ㅂㅗ-ㅇㅣ-ㄱㅣ-ㅁㅏㄴ ㅎㅐㅆㄷㅓㄴ ㅋㅓ-ㅅㅡ-ㅌㅣㄴ ㄷㅓㄴㅅㅡ-ㅌㅡ-ㄱㅏ- ㄴㅓ-ㅁㅜ-ㄴㅏ-ㄷㅗ- ㅇㅣ-ㅃㅓ-ㅂㅗ-ㅇㅕㅆㄷㅏ-']

In [146]:
corpus_train = train[:120000]
corpus_test = train[120000:]

with open('navermovie_fasttext_train.txt', 'w') as f:
    for txt in corpus_train:
        f.write(txt+'\n')
        
with open('navermovie_fasttext_test.txt', 'w') as f:
    for txt in corpus_test:
        f.write(txt+'\n')

In [147]:
corpus_path_train = 'navermovie_fasttext_train.txt'
corpus_path_test = 'navermovie_fasttext_test.txt'

### word representation using subword

In [148]:
# import config
from soynlp.hangle import decompose, compose
from soynlp.normalizer import remove_doublespace

## lovit님의 function

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in s.split())

# 초/중/종성을 나눴기 때문에 cosine similarity을 계산할 때에도 입력될 단어를 초/중/종성으로 나눈다.
def cosine_similarity(word1, word2):
    word1 = encode(word1)
    word2 = encode(word2)
    return embedding_model.cosine_similarity(word1, word2)

In [149]:
fasttext_corpus[0]

['ㅇㅏ-', 'ㄷㅓ-ㅂㅣㅇ', 'ㅈㅣㄴㅉㅏ-', 'ㅉㅏ-ㅈㅡㅇㄴㅏ-ㄴㅔ-ㅇㅛ-', 'ㅁㅗㄱㅅㅗ-ㄹㅣ-']

### Model Building
* load_model 을 할 때에는 fasttext_model_name 뒤에 확장자 '.bin' 을 붙여줘야 한다. .bin 과 .vec 두 가지의 파일을 만들기 때문이다.

In [150]:
# 데이터 형태
with open(corpus_path_train) as f:
    for _ in range(3):
        print(next(f).strip()[:50])

__label__neg ㅇㅏ- ㄷㅓ-ㅂㅣㅇ ㅈㅣㄴㅉㅏ- ㅉㅏ-ㅈㅡㅇㄴㅏ-ㄴㅔ-ㅇㅛ- ㅁㅗㄱ
__label__pos ㅎㅡㅁ ㅍㅗ-ㅅㅡ-ㅌㅓ-ㅂㅗ-ㄱㅗ- ㅊㅗ-ㄷㅣㅇㅇㅕㅇㅎㅘ-ㅈㅜㄹ ㅇ
__label__neg ㄴㅓ-ㅁㅜ-ㅈㅐ-ㅁㅣㅎㅇㅓㅆㄷㅏ-ㄱㅡ-ㄹㅐ-ㅅㅓ-ㅂㅗ-ㄴㅡㄴㄱㅓㅅㅇ


In [151]:
# supervised model
model = fasttext.train_supervised("navermovie_fasttext_train.txt")
model.save_model("model_file.bin")

In [152]:
sent = '영화 대박 크리스토ㅍㅓ 연기 진짜 좋아'

words = encode(sent).split()
model.predict(words,k=2)

([['__label__pos', '__label__neg'],
  ['__label__pos', '__label__neg'],
  ['__label__pos', '__label__neg'],
  ['__label__pos', '__label__neg'],
  ['__label__neg', '__label__pos'],
  ['__label__pos', '__label__neg']],
 [array([0.50978833, 0.49023172], dtype=float32),
  array([0.994162  , 0.00585799], dtype=float32),
  array([0.88849556, 0.11152444], dtype=float32),
  array([0.91063535, 0.08938462], dtype=float32),
  array([0.53648657, 0.46353343], dtype=float32),
  array([0.99560976, 0.0044102 ], dtype=float32)])

In [153]:
def final_prob(pred):
    label = pred[0]
    prob = pred[1]
    n = len(label)
    
    prob_pos = []
    prob_neg = []
    
    for j in range(n):
        tmp_pos = [True if i == '__label__pos' else False for i in label[j]]
        tmp_neg = [True if i == '__label__neg' else False for i in label[j]]
        pos_idx = np.where(tmp_pos)[0]
        neg_idx = np.where(tmp_neg)[0]
        
        prob_pos.append(prob[j][pos_idx])
        prob_neg.append(prob[j][neg_idx])
        
    return sum(prob_pos)/n, sum(prob_neg)/n

final_prob(model.predict(words,k=2))

(array([0.7937041], dtype=float32), array([0.20631592], dtype=float32))

### evaluate model with test data

In [157]:
with open('navermovie_fasttext_test.txt', 'r') as f:
    test_review = f.readlines()

In [195]:
actual = []
pred = []
err_idx = []
a = 120000

for review in test_review:
    review = review[:-2]
    tmp = review.split(' ')
    actual.append( tmp[0][-3:] )
    
    words = tmp[1:]
    
    try:
    
        result = model.predict(words, k=2)

        if final_prob(result)[0][0] > final_prob(result)[1][0]:
            pred.append('pos')
        else:
            pred.append('neg')
            
    except:
        pred.append('pos')
        err_idx.append(a)
        
    a+=1

In [202]:
# 비어있는 리뷰. 얘네들은 제외한다.
for i,txt in enumerate(test_review):
    if i+120000 in err_idx:
        print(txt)

__label__pos 

__label__neg 

__label__pos 

__label__neg 

__label__neg 

__label__pos 

__label__pos 

__label__neg 

__label__pos 

__label__pos 

__label__neg 

__label__pos 

__label__neg 

__label__pos 

__label__pos 

__label__neg 

__label__neg 

__label__pos 

__label__neg 

__label__neg 

__label__neg 

__label__neg 

__label__pos 

__label__pos 

__label__neg 

__label__neg 

__label__neg 

__label__neg 

__label__neg 

__label__pos 

__label__neg 

__label__neg 

__label__pos 

__label__pos 

__label__neg 

__label__pos 

__label__pos 

__label__neg 

__label__neg 

__label__neg 

__label__neg 

__label__neg 

__label__pos 

__label__neg 

__label__neg 

__label__pos 

__label__neg 

__label__pos 

__label__neg 

__label__pos 

__label__pos 

__label__neg 

__label__neg 

__label__pos 

__label__neg 

__label__pos 

__label__neg 

__label__neg 

__label__pos 

__label__pos 

__label__neg 

__label__pos 

__label__neg 

__label__neg 

__label__pos 

__label__neg 

__label__n

In [206]:
actual = [j for i,j in enumerate(actual) if i+120000 not in err_idx]
pred = [j for i,j in enumerate(pred) if i+120000 not in err_idx]

In [210]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(actual,pred)

array([[10244,  4672],
       [ 2556, 12188]])

In [211]:
print(classification_report(actual, pred))

              precision    recall  f1-score   support

         neg       0.80      0.69      0.74     14916
         pos       0.72      0.83      0.77     14744

    accuracy                           0.76     29660
   macro avg       0.76      0.76      0.76     29660
weighted avg       0.76      0.76      0.76     29660

